# Khipus.ai
## Embeddings
### Demo: Building a Business‑Intelligence Agent with Function Tools
<span>© Copyright Notice 2025, Khipus.ai - All Rights Reserved.</span>



This notebook shows how to build a small *business‑intelligence* agent that can:

* fetch live stock prices,  
* look up foreign‑exchange (FX) rates,  
* retrieve the latest company news, and  
* weave the results into natural‑language answers with an LLM.

We'll focus on the concept of **agent tools**—special Python functions the language‑model can call when it needs structured data.

## Learning Objectives
By the end of this lab you should be able to:
1. Write a function‑tool that returns JSON‑serialisable data.
2. Register tools with an `Agent` so the LLM can decide which ones to call.
3. Combine multiple tool calls in a single response.
4. Extend the agent with your own custom data sources.

> ⚙️ **Setup**

This notebook assumes:

* Python ≥ 3.9  
* Packages: `openai`, `yfinance`, `agents`, `rich`, `python-dotenv`

If you are missing any of them, uncomment and run the next cell.


In [16]:
# !pip install --quiet openai yfinance agents rich python-dotenv
#%pip install yfinance


## 1 — Configure your API keys
We will use:
* **Azure OpenAI** for the LLM.
* **Yahoo Finance** (via the `yfinance` library) for market data.

Set the following environment variables **before** running the rest of the notebook:
```bash
export ENDPOINT_URL="https://khipusaigpt0566189501.openai.azure.com/"
export DEPLOYMENT_NAME="gpt-4o"
export AZURE_OPENAI_API_KEY="Your key" 
```

The notebook will use Azure OpenAI with the specified endpoint and deployment.

In [ ]:
import os, logging, asyncio
from datetime import datetime, timezone, timedelta

import openai, yfinance as yf
from agents import Agent, OpenAIChatCompletionsModel, Runner, function_tool, set_tracing_disabled
from rich.logging import RichHandler

# ----- Logging (pretty console output) -----
logging.basicConfig(level=logging.INFO, format="%(message)s", handlers=[RichHandler()])

# Disable OpenAI tracing if not configured
set_tracing_disabled(True)

# ----- Azure OpenAI Configuration -----
endpoint = os.getenv("ENDPOINT_URL", "https://khipusaigpt0566189501.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY", "Your key") # Ask your instructor for the key
api_version = "2025-01-01-preview"

client = openai.AsyncAzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
)

MODEL_NAME = deployment

## 2 — Define our data‑retrieval tools
Each tool is a normal Python function annotated with `@function_tool` so that the `Agent` knows it can be invoked.

In [18]:
def _today_iso() -> str:
    return datetime.now(timezone.utc).strftime("%Y-%m-%d")


@function_tool
def get_stock_price(symbol: str) -> dict:
    """Return the latest share price from Yahoo Finance."""
    tk = yf.Ticker(symbol.upper())
    price = tk.fast_info.get("last_price") or tk.history(period="1d")["Close"].iloc[-1]
    currency = tk.fast_info.get("currency", "USD")
    return {"symbol": symbol.upper(), "price": round(float(price), 2), "currency": currency, "date": _today_iso()}


@function_tool
def get_fx_rate(pair: str) -> dict:
    """Return the latest FX rate for a six‑letter currency pair (e.g. EURUSD)."""
    pair = pair.upper()
    if len(pair) != 6:
        raise ValueError("Pair must be six letters, e.g. EURUSD")
    tk = yf.Ticker(f"{pair}=X")
    rate = tk.fast_info.get("last_price") or tk.history(period="1d")["Close"].iloc[-1]
    return {"pair": pair, "rate": round(float(rate), 4), "date": _today_iso()}


@function_tool
def get_company_news(company: str, days: int = 10) -> list:
    """Return news headlines for the given company from Yahoo Finance."""
    tk = yf.Ticker(company)
    cutoff = datetime.now(timezone.utc) - timedelta(days=days)
    headlines = []
    for item in tk.news or []:
        ts = datetime.fromtimestamp(item["providerPublishTime"], tz=timezone.utc)
        if ts >= cutoff:
            headlines.append({
                "title": item["title"],
                "publisher": item["publisher"],
                "published": ts.strftime("%Y-%m-%d"),
                "link": item["link"],
            })
    return headlines or [f"No significant news in last {days} days."]


@function_tool
def get_current_date() -> str:
    return _today_iso()


## 3 — Create the Agent
We wire our tools into a single agent with a short system prompt.

In [19]:
business_agent = Agent(
    name="Business Analyst",
    instructions=(
        "You are a senior business‑intelligence assistant. " 
        "Provide concise, data‑backed answers on markets, finance, and corporate strategy. " 
        "When citing numbers, include the retrieval date."
    ),
    tools=[get_stock_price, get_fx_rate, get_company_news, get_current_date],
    model=OpenAIChatCompletionsModel(model=MODEL_NAME, openai_client=client),
)


## 4 — Ask a question

In [20]:
async def demo():
    prompt = "Give me today's MSFT share price, the EUR‑USD rate, and any notable Microsoft news."
    result = await Runner.run(business_agent, input=prompt)
    print(result.final_output)

await demo()


[07/14/25 10:54:52] INFO     HTTP Request: POST                                                     ]8;id=717259;file://c:\temp\python-ai-agent-frameworks-demos\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=638076;file://c:\temp\python-ai-agent-frameworks-demos\.venv\Lib\site-packages\httpx\_client.py#1740\1740]8;;\
                             https://khipusaigpt0566189501.openai.azure.com/openai/deployments/gpt-                
                             4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"                  

[07/14/25 10:54:54] INFO     HTTP Request: POST                                                     ]8;id=390940;file://c:\temp\python-ai-agent-frameworks-demos\.venv\Lib\site-packages\httpx\_client.py\_client.py]8;;\:]8;id=84280;file://c:\temp\python-ai-agent-frameworks-demos\.venv\Lib\site-packages\httpx\_client.py#1740\1740]8;;\
                             https://khipusaigpt0566189501.openai.azure.com/openai/deployments/gpt-                
                             4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"                  

**Microsoft (MSFT) Share Price:** $502.87 (retrieved on 2025-07-14).  
**EUR/USD Exchange Rate:** 1.1677 (retrieved on 2025-07-14).  
**Microsoft News:** No notable news has been reported in the last day.


## 5 — How it works
1. **The LLM analyses the prompt** and decides it needs structured data.
2. **Tool calls** are automatically triggered:
   * `get_stock_price` → JSON with symbol, price, currency, date.
   * `get_fx_rate` → JSON with pair, rate, date.
   * `get_company_news` → list of headline dictionaries.
3. **The agent stitches** those payloads into a fluent, human‑readable answer.

Because each tool returns plain dictionaries/lists, you could as easily connect to a database, a REST endpoint, or a sensor.

## 6 — Your Turn ✏️
* **Exercise A:** Add a new tool named `get_crypto_price(coin: str)` that looks up Bitcoin, Ethereum, etc.
* **Exercise B:** Modify the prompt so the agent also gives a brief strategic recommendation based on the data.
* **Exercise C:** Swap `Runner.run` for `Runner.interact()` to get an interactive chat session.